<center>
    <h1><u>MySQL Agent</u></h1>
</center>


For this project we will use the employees.sql sample database. which can be downloaded from here: https://github.com/datacharmer/test_db

Also, I use a LOCAL model, hence, completely free to use.
In this example I use Ollama with a local model. You could use vLLM, LM Studio, or any other LLM GUI service.

In this project, we will use the following:
1. Engine - establishing the connection between Python and the MySQL database.
2. Toolkit - providing tools for the Agent to interact with the database.
3. Agent - reasoning and decision-making.

---


<h2>Instructions</h2>

1. Install mysql server on your machine (depends on OS) : https://dev.mysql.com/doc/mysql-installation-excerpt/5.7/en/

2. Open terminal and run the following commands:

   a. `git clone https://github.com/datacharmer/test_db`

   b. `cd test_db`

   c. `mysql -u root -p < employees.sql`

* Note: I used a docker container to run the mysql server, but you can use the local server as well using the steps above.


---


<center> 
    <h2>Imports</h2>
</center>


In [1]:
# LLM Imports
from langchain_ollama import OllamaLLM
from langchain_community.agent_toolkits import create_sql_agent

# Custom Prompt Imports
from langchain.agents.agent_types import AgentType
from langchain.prompts import PromptTemplate

# MySQL Imports
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from sqlalchemy import create_engine


# Environment Variables
from dotenv import load_dotenv
import os

# Other Imports
import pandas as pd
from IPython.display import display, Markdown

load_dotenv()

True

---


<center> 
    <h2>Load Environment Variables</h2>
</center>


* Created a .env file in the root directory of the project
* Added the following variables:
```
MYSQL_HOST=
MYSQL_PORT=
MYSQL_USER=
MYSQL_PASSWORD=
MYSQL_DB=
MODEL=
```

* I used a NON-REASONING model, since my sql_agent does the reasoning for me.
* The model I used is `llama3.2` (3B parameters version)
* Therefore, not need to use a reasoning model like deepseek-ai/DeepSeek-R1.

In [2]:
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT")
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_DB = os.getenv("MYSQL_DB")
MODEL = os.getenv("MODEL")

In [3]:
MODEL

'llama3.2'

In [4]:
SQL_URI = f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"

---


<center> 
    <h2>Build the SQL Database</h2>
</center>


## 1. Engine
* In this context, an engine refers to a database engine used to establish a connection between Python and the database.
* The function create_engine(SQL_URI) comes from SQLAlchemy, which is a Python library used for interacting with relational databases.
* The engine handles connections, transactions, and query execution.

In [5]:
# Create the SQLAlchemy engine first
engine = create_engine(SQL_URI)

In [6]:
# Connect to your MySQL database
db = SQLDatabase(engine)

In [7]:
# Test database connection
db.get_usable_table_names()

['departments', 'dept_emp', 'dept_manager', 'employees', 'salaries', 'titles']

In [8]:
# Get the first table in human readable format
tables = db.get_usable_table_names()
for table in tables[:1]:
    # Convert query results to pandas DataFrames
    schema_df = pd.read_sql(f"DESCRIBE {table}", engine)
    sample_df = pd.read_sql(f"SELECT * FROM {table} LIMIT 3", engine)

    print(f"\nSchema for table {table}:")
    display(schema_df)
    print(f"\nSample Data for table {table}:")
    display(sample_df)


Schema for table departments:


,Field,Type,Null,Key,Default,Extra
0,dept_no,char(4),NO,PRI,None,
1,dept_name,varchar(40),NO,UNI,None,



Sample Data for table departments:


,dept_no,dept_name
0,d009,Customer Service
1,d005,Development
2,d002,Finance


---


<center> 
    <h2>Configure the LLM</h2>
</center>


### If you wish to use vLLM, follow these steps:

1. inside the terminal run `vllm serve <model_name> --tensor-parallel-size 2 --max-model-len 32768 --enforce-eager`

   for example: `vllm serve deepseek-ai/DeepSeek-R1 --tensor-parallel-size 2 --max-model-len 32768 --enforce-eager`

##### <u>For CPU usage:</u>

`vllm serve deepseek-ai/DeepSeek-R1 --tensor-parallel-size 1 --enforce-eager --device cpu --trust-remote-code`

##### <u>for GPU usage:</u>

`vllm serve deepseek-ai/DeepSeek-R1 --tensor-parallel-size 1 --enforce-eager --device cuda --trust-remote-code`


---


### Note: I will not use vLLM, instead I will use ollama with a local model.

1. In order to use ollama, you need to install it first: https://ollama.ai/
2. Then run the following command: `ollama run <model_name>` <br> for example: <br> `ollama run deepseek-ai/DeepSeek-R1`
<br>
or
<br>
`ollama run llama3.2`

- You can find the list of models here: https://ollama.ai/models
- Make sure you know your computer's capacity for LLM models (CPU, GPU, etc.)


In [9]:
# Configure the LLM to use vLLM's OpenAI-compatible endpoint
model = OllamaLLM(
    base_url="http://localhost:11434",
    model=MODEL,
    temperature=0.1,
    top_k=10,
    top_p=0.95,  # Add top_p for better sampling
    repeat_penalty=1.1,  # repeat penalty helps to avoid repetitive outputs
)

## 2. Toolkit
* A toolkit is a collection of helper functions or utilities that an Agent can use to accomplish specific tasks. such as running queries, retrieving results, etc. (the actual `run` function for example when the agent wants to run a query)
* In the case of LangChain, the SQL toolkit provides tools that allow an AI agent to interact with a SQL database, run queries, and retrieve results.
* The toolkit doesn't execute queries by itself—it provides structured ways for an Agent to interact with a database.

In [10]:
# Create the SQL toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=model)

<center> 
    <h2>Create the SQL Agent</h2>
</center>


## 3. Agent
* An Agent is responsible for **reasoning** and **decision-making**. It takes user input (natural language) and decides how to process it.
* In this case, the Agent converts natural language into SQL queries using an LLM and executes them on the database.
* It decides which tools to use, meaning it calls the SQL toolkit when needed.

In [11]:
# Create a custom prompt template for the agent
custom_prompt = PromptTemplate(
    input_variables=["input"],
    template="""You are a SQL expert. Your task is to:
1. Convert the user's question into a SQL query
2. Execute that query using the provided tools
3. Return the actual results in a simple sentence

Important: When you see sample data in table schemas, ignore the number of rows shown - they are just examples.

User Question: {input}

Let's solve this step by step:""",
)

In [12]:
# Create the SQL agent
agent_executor = create_sql_agent(
    llm=model,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Depending on the model, you can change the agent type
    # For example, if you want to use an OpenAI model, you can use: AgentType.OPENAI_MULTI_FUNCTIONS
    max_iterations=None,  # Depending on the length of the query, you can change the max iterations
    early_stopping_method="force",
    return_intermediate_steps=False,
    agent_executor_kwargs={
        "handle_parsing_errors": True  # Make sure to use this syntax (dictionary) and not the simple handle_parsing_errors=True parameter
    },
)

---


<center> 
    <h2>Prompts</h2>
</center>


In [13]:
# Create a wrapper function for better responses, just optional, you could use response["output"]
def ask_database(question: str) -> str:
    try:
        response = agent_executor.invoke({"input": question})
        # Extract just the final answer if it's in the response
        if isinstance(response, dict) and "output" in response:
            return response["output"]
        return str(response)
    except Exception as e:
        return f"Error: {str(e)}"

In [14]:
result = ask_database("How many employees are there in total?")



> Entering new SQL Agent Executor chain...


Action: sql_db_list_tables
Action Input: departments, dept_emp, dept_manager, employees, salaries, titlesQuestion: How many employees are there in total?

Thought: Since we need to find the total number of employees, I should query the 'employees' table.

Action: sql_db_schema
Action Input: employees
CREATE TABLE employees (
	emp_no INTEGER NOT NULL, 
	birth_date DATE NOT NULL, 
	first_name VARCHAR(14) NOT NULL, 
	last_name VARCHAR(16) NOT NULL, 
	gender ENUM('M','F') NOT NULL, 
	hire_date DATE NOT NULL, 
	PRIMARY KEY (emp_no)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from employees table:
emp_no	birth_date	first_name	last_name	gender	hire_date
10001	1953-09-02	Georgi	Facello	M	1986-06-26
10002	1964-06-02	Bezalel	Simmel	F	1985-11-21
10003	1959-12-03	Parto	Bamford	M	1986-08-28
*/Question: How many employees are there in total?
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relev

In [15]:
print(result)

The total number of employees is 300024.
